### Load dataset

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

#os.environ.get("OPENAI_API_KEY")

True

In [11]:
category_column = "category"
answer_column = "answer"
embedded_file = "embeds.csv"
results_file = "results.csv"

In [12]:
import pandas as pd
import numpy as np


# Currenty pre-processed - empty catergoies and answers removed
df = pd.read_csv("./questions.csv", sep=";")

#Remove rows with empty answer_column or category_column values
#df[[category_column, answer_column]].replace('', np.nan, inplace=True)
#df = df[[category_column, answer_column]].dropna()

#df = df.loc[pd.notnull(df[answer_column])]

df = df.reset_index(drop=True)

df.head(3)

,category,question,answer
0,Restaurant,What is the name of the restaurant?,The name of the restaurant is 'Pizza Fun'.
1,Dish,What is a popular dish at Pizza Fun?,The most popular dish at Pizza Fun is their 'S...
2,Ingredients,NaN,NaN


In [13]:
df.columns

Index(['category', 'question', 'answer'], dtype='object')

In [14]:
categories = sorted(df[category_column].unique())
categories

['Ambiance', 'Dish', 'Ingredients', 'Opening times', 'Restaurant']

In [15]:
from openai.embeddings_utils import get_embeddings
matrix = get_embeddings(df[answer_column].to_list(), engine="text-embedding-ada-002")

RetryError: RetryError[<Future at 0x1bac6940450 state=finished raised AttributeError>]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
vis_dims = pca.fit_transform(matrix)
#TODO - divide dimensions into their own columns
df["embed_vis"] = vis_dims.tolist()

df1 = pd.DataFrame( vis_dims.tolist())

df1.columns = ['text_emb_dim1_' + answer_column, 'text_emb_dim2_'+ answer_column, 'text_emb_dim3_'+ answer_column]

results = df.join(df1)

In [ ]:
import plotly.graph_objs as go
import numpy as np

fig = go.Figure()

for i, cat in enumerate(categories):
    sub_matrix = np.array(df[df["category"] == cat]["embed_vis"].to_list())
    x = sub_matrix[:, 0]
    y = sub_matrix[:, 1]
    z = sub_matrix[:, 2]

    fig.add_trace(
        go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode="markers",
            marker=dict(size=5, color=i, colorscale="Viridis", opacity=0.8),
            name=cat,
        )
    )

fig.update_layout(
    autosize=False,
    title="3D Scatter Plot of Categories",
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=10),
    scene=dict(
        xaxis=dict(title="x"),
        yaxis=dict(title="y"),
        zaxis=dict(title="z"),
    ),
)

fig.show()

In [ ]:
#write out embeds
df1.to_csv(embedded_file, sep='\t')
#write out input file with embeds
results.drop('embed_vis', axis=1, inplace=True)
#results.reset_index(drop=True, inplace=True)
results.to_csv(results_file, sep='\t', index=False)